# Daten der KMK von PDFs mit eingescannten Seiten von 1984, 1990 und 1995

Die KMK Daten der Jahre 1984, 1990 und 1995 wurden per hand aus den PDFs in Exceltabellen übertragen.

Folgende Bibliotheken werden geladen zur Analyse

In [2]:
import pandas as pd
import numpy as np
import re

Aus den Excel Dateien werden die Daten in Pandas DataFrames geladen. Ein Format das wie eine mini rationale Datenbank zu handhaben ist.

In [5]:
data84 = pd.read_excel('Data/input/KMK_1984.xlsx')
data90 = pd.read_excel('Data/input/KMK_1990.xlsx')
data95 = pd.read_excel('Data/input/KMK_1995.xlsx')

Mit dem command head() kann man die ersten 5 Zeilen eines DataFrames sehen:

In [6]:
data84.head()

,Beruf,Land,besuchendesLand,Unnamed: 3
0,Bergmechaniker (I),"Saarland, Camphausen","BW,SA",NaN
1,Betonstein- und Terrazzohersteller (Hw/I),"Baden-Württemberg, Ulm","BW,BR,HA,HE,NS,RP, SA,SH",NaN
2,Betonwerker (I),"Baden-Württemberg, Ulm","BW,BR,HA,HE,NS, RP,SA,SH",NaN
3,Binnenschiffer (I),"Nordrhein-Westfalen, Petershagen","BW, BR,NS,NW,RP,SH",NaN
4,Binnenschiffer (I),"Nordrhein-Westfalen, Homberg","BW, BR,NS,NW,RP,SH",NaN


In [64]:
landsShortList = ['BB','BE','BY','BW','MV','SN','ST','SL','SH','TH','HB','HE','HH','NI','NW','RP','Alle']
len(landsShortList)
landsShortList = sorted(landsShortList)

In ein "array" werden im folgenden die unique entries aus der Spalte 'Land' mit Hilfe des commands unique() geladen.

In [12]:
uniqueLands84 = data84['Land'].unique()
uniqueLands90 = data90['Land'].unique()
uniqueLands95 = data95['Land'].unique()

In [13]:
uniqueLands84[0:4]

array(['Saarland, Camphausen', 'Baden-Württemberg, Ulm',
       'Nordrhein-Westfalen, Petershagen', 'Nordrhein-Westfalen, Homberg'], dtype=object)

In [18]:
print('==== Format der 1990er Daten =====')
print(uniqueLands90[0:4])
print('==== Format der 1995er Daten =====')
print(uniqueLands95[0:4])

==== Format der 1990er Daten =====
['NW, Essen' 'BW, Heidelberg' 'HB, Bremen' 'HH, Hamburg']
==== Format der 1995er Daten =====
['Nordrhein Westfalen' 'Mecklenburg-Vorpommern' 'Baden-Württemberg'
 'Thüringen']


Die Spalte 'Land' enthält teilweise Komma separiert Land und Stadt als Eintrag. Da ein paar Wochen zwischen dem Erhalt der Daten lag, ist leider dieser Fehler beim Übertrag in Excel aufgetreten. Dies wird im folgenden bereinigt.

Zusätzlich wird mit dem Kommand info() schnell überprüft ob es weitere Fehler gibt.

In [22]:
print(data84.info())
print(data90.info())
print(data95.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 4 columns):
Beruf              160 non-null object
Land               160 non-null object
besuchendesLand    160 non-null object
Unnamed: 3         1 non-null object
dtypes: object(4)
memory usage: 5.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 3 columns):
Beruf              174 non-null object
Land               174 non-null object
besuchendesLand    174 non-null object
dtypes: object(3)
memory usage: 4.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 3 columns):
Beruf              336 non-null object
Land               336 non-null object
besuchendesLand    336 non-null object
dtypes: object(3)
memory usage: 8.0+ KB
None


In [23]:
len(uniqueLands95)

19

In [8]:
def cleanLands1995(land):
    if land == 'Schleswig-Holstein ':
        land = 'Schleswig-Holstein'
    if land == 'berlin':
        land = 'Berlin'
    if land == 'sachsen':
        land = 'Sachsen'
    else:
        land = land

    return land

In [69]:
def shortTOLongNamer(short):
    longName = ' '
    if short == 'NW':
        longName = 'Nordrhein-Westfalen'
    elif short == 'BW':
        longName = 'Baden-Württemberg'
    elif short == 'BY':
        longName = 'Bayern'
    elif short == 'HB':
        longName = 'Bremen'
    elif short == 'HH':
        longName = 'Hamburg'
    elif short == 'HE':
        longName = 'Hessen'
    elif short == 'RP':
        longName = 'Rheinland-Pfalz'
    elif short == 'SH':
        longName = 'Schleswig-Holstein'
    elif short == 'SL':
        longName = 'Saarland'
    elif short == 'NS':
        longName = 'Niedersachsen'
    elif short == 'BE':
        longName = 'Berling'
    else:
        longName = short
    return longName

In [70]:
uniques1984 = ['Baden-Würtemberg',
 'Baden-Württemberg',
 'Bayern',
 'Hamburg',
 'Hessen',
 'Niedersachsen',
 'Nordrhein-Westfalen',
 'Nordrhein-westfalen',
 'Rheinladn-Pfalz',
 'Rheinland-Pfalz',
 'Rheinland-Pflaz',
 'Saarland',
 'Schleswig-Holstein']
def cleanLands1984(land):
    if land == 'Baden-Würtemberg':
        land = 'Baden-Württemberg'
    if land == 'Rheinladn-Pfalz':
        land = 'Rheinland-Pfalz'
    if land == 'Rheinland-Pflaz':
        land = 'Rheinland-Pfalz'
    if land == 'Nordrhein-westfalen':
        land = 'Nordrhein-Westfalen'
    else:
        land = land

    return land

In [72]:
landList = []
stadtList =[]
for l in land:
    s = l
    print(l)
#     print(s)
    k = re.split(string=s, pattern=',')
    longName = shortTOLongNamer(k[0].upper())
#     longName = cleanLands1995(l)
#     longName = cleanLands1984(k[0])
    landList.append(longName)
    stadtList.append(k[1])    
    print (longName)

NW, Essen
Nordrhein-Westfalen
BW, Heidelberg
Baden-Württemberg
HB, Bremen
Bremen
HH, Hamburg
Hamburg
BY, Selb
Bayern
SL, Quierschied-Camphausen
Saarland
BW, Ulm
Baden-Württemberg
NW, Petershagen
Nordrhein-Westfalen
NW, Duisburg-Homburg
Nordrhein-Westfalen
BY, Coburg
Bayern
SH, Lübeck
Schleswig-Holstein
BW, Ulm
Baden-Württemberg
BY, Karlstadt
Bayern
HB, Bremen
Bremen
NW, Dortmund
Nordrhein-Westfalen
NW, Köln
Nordrhein-Westfalen
BY, Zwiesel
Bayern
NS, Bad Zwischenahn-Rotstrup
Niedersachsen
BW, Ehingen
Baden-Württemberg
BY, Rothenburg o.d.T.
Bayern
BW, Tuttlingen
Baden-Württemberg
NW, Köln
Nordrhein-Westfalen
RP, Idar-Oberstein
Rheinland-Pfalz
BY, Neustadt Saale
Bayern
HE, Michelstadt
Hessen
By, Schweinfurt
Bayern
HE, Geisenheim
Hessen
BW, Heilbronn
Baden-Württemberg
HB, Bremen
Bremen
SH, Neumünster
Schleswig-Holstein
NW, Solingen
Nordrhein-Westfalen
BY, Zwiesel
Bayern
HE, Wetzlar
Hessen
BW, Pforzheim
Baden-Württemberg
NW, Bielefeld
Nordrhein-Westfalen
NW, Gelsenkirchen
Nordrhein-Westfale

In [73]:
landList

['Nordrhein-Westfalen',
 'Baden-Württemberg',
 'Bremen',
 'Hamburg',
 'Bayern',
 'Saarland',
 'Baden-Württemberg',
 'Nordrhein-Westfalen',
 'Nordrhein-Westfalen',
 'Bayern',
 'Schleswig-Holstein',
 'Baden-Württemberg',
 'Bayern',
 'Bremen',
 'Nordrhein-Westfalen',
 'Nordrhein-Westfalen',
 'Bayern',
 'Niedersachsen',
 'Baden-Württemberg',
 'Bayern',
 'Baden-Württemberg',
 'Nordrhein-Westfalen',
 'Rheinland-Pfalz',
 'Bayern',
 'Hessen',
 'Bayern',
 'Hessen',
 'Baden-Württemberg',
 'Bremen',
 'Schleswig-Holstein',
 'Nordrhein-Westfalen',
 'Bayern',
 'Hessen',
 'Baden-Württemberg',
 'Nordrhein-Westfalen',
 'Nordrhein-Westfalen',
 'Bayern',
 'Bayern',
 'Bayern',
 'Niedersachsen',
 'Niedersachsen',
 'Schleswig-Holstein',
 'Schleswig-Holstein',
 'Baden-Württemberg',
 'Hessen',
 'Baden-Württemberg',
 'Nordrhein-Westfalen',
 'Baden-Württemberg',
 'Baden-Württemberg',
 'Bayern',
 'Baden-Württemberg',
 'Bayern',
 'Baden-Württemberg',
 'Bayern',
 'Hessen',
 'Bayern',
 'Bayern',
 'Bayern',
 'Bayern

In [74]:
data200['Land'] = landList
# data200['Stadt'] = stadtList

In [75]:
data200.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 3 columns):
Beruf              174 non-null object
Land               174 non-null object
besuchendesLand    174 non-null object
dtypes: object(3)
memory usage: 4.2+ KB


In [76]:
data200.head()

,Beruf,Land,besuchendesLand
0,Asphaltbauer (I),Nordrhein-Westfalen,alle Länder
1,Bandagist (Hw),Baden-Württemberg,BW RP SA
2,Bandagist (Hw),Bremen,BR NS
3,Bandagist (Hw),Hamburg,HA NS SH
4,Baustoffprüfer (I),Bayern,BW BY HA HE NS RP SA SH


In [77]:
data200['Berlin'] = data200['besuchendesLand'].str.contains('BE')
data200['Brandenburg'] = data200['besuchendesLand'].str.contains('BB')
data200['Bayern'] = data200['besuchendesLand'].str.contains('BY')
data200['Baden-Württemberg'] = data200['besuchendesLand'].str.contains('BW')
data200['Mecklenburg-Vorpommern'] = data200['besuchendesLand'].str.contains('MV')
data200['Sachsen'] = data200['besuchendesLand'].str.contains('SN')
data200['Sachsen-Anhalt'] = data200['besuchendesLand'].str.contains('ST')
data200['Saarland'] = data200['besuchendesLand'].str.contains('SL')
data200['Saarland'] = data200['besuchendesLand'].str.contains('SA')
data200['Schleswig-Holstein'] = data200['besuchendesLand'].str.contains('SH')
data200['Thüringen'] = data200['besuchendesLand'].str.contains('TH')
data200['Bremen'] = data200['besuchendesLand'].str.contains('HB')
data200['Bremen'] = data200['besuchendesLand'].str.contains('BR')
data200['Hessen'] = data200['besuchendesLand'].str.contains('HE')
data200['Hamburg'] = data200['besuchendesLand'].str.contains('HH')
data200['Hamburg'] = data200['besuchendesLand'].str.contains('HA')
data200['Niedersachsen'] = data200['besuchendesLand'].str.contains('NI')
data200['Niedersachsen'] = data200['besuchendesLand'].str.contains('NS')
data200['Nordrhein-Westfalen'] = data200['besuchendesLand'].str.contains('NW')
data200['Rheinland-Pfalz'] = data200['besuchendesLand'].str.contains('RP')
data200['Alle'] = data200['besuchendesLand'].str.contains('alle')

In [78]:
data200.head()

,Beruf,Land,besuchendesLand,Berlin,Brandenburg,Bayern,Baden-Württemberg,Mecklenburg-Vorpommern,Sachsen,Sachsen-Anhalt,Saarland,Schleswig-Holstein,Thüringen,Bremen,Hessen,Hamburg,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Alle
0,Asphaltbauer (I),Nordrhein-Westfalen,alle Länder,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,Bandagist (Hw),Baden-Württemberg,BW RP SA,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False
2,Bandagist (Hw),Bremen,BR NS,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
3,Bandagist (Hw),Hamburg,HA NS SH,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False
4,Baustoffprüfer (I),Bayern,BW BY HA HE NS RP SA SH,False,False,True,True,False,False,False,True,True,False,False,True,True,True,False,True,False


In [79]:
len(landList)

174

In [80]:
uniqueLands = data200['Land'].unique()

In [81]:
sortedUniqueLands = sorted(uniqueLands)
sortedUniqueLands

['Baden-Württemberg',
 'Bayern',
 'Bremen',
 'Hamburg',
 'Hessen',
 'Niedersachsen',
 'Nordrhein-Westfalen',
 'Rheinland-Pfalz',
 'Saarland',
 'Schleswig-Holstein']

In [82]:
dictLands = {}
for land in uniqueLands:
    dictLands[land] = data200[data200['Land'] == land]['Land'].count()

In [83]:
dictLands

{'Baden-Württemberg': 38,
 'Bayern': 58,
 'Bremen': 4,
 'Hamburg': 7,
 'Hessen': 8,
 'Niedersachsen': 11,
 'Nordrhein-Westfalen': 28,
 'Rheinland-Pfalz': 9,
 'Saarland': 3,
 'Schleswig-Holstein': 8}

In [84]:
landsShortList

['Alle',
 'BB',
 'BE',
 'BW',
 'BY',
 'HB',
 'HE',
 'HH',
 'MV',
 'NI',
 'NW',
 'RP',
 'SH',
 'SL',
 'SN',
 'ST',
 'TH']

In [85]:
dfLaender = pd.DataFrame(data = sorted(uniqueLands), columns=['Land'])
dfLaender

,Land
0,Baden-Württemberg
1,Bayern
2,Bremen
3,Hamburg
4,Hessen
5,Niedersachsen
6,Nordrhein-Westfalen
7,Rheinland-Pfalz
8,Saarland
9,Schleswig-Holstein


In [86]:
count = np.ones(10, dtype=np.int32)
for i,land in enumerate(sorted(uniqueLands)):
    count[i] = data200[data200['Land'] == land]['Land'].count()
    print(i,land)
count

0 Baden-Württemberg
1 Bayern
2 Bremen
3 Hamburg
4 Hessen
5 Niedersachsen
6 Nordrhein-Westfalen
7 Rheinland-Pfalz
8 Saarland
9 Schleswig-Holstein


array([38, 58,  4,  7,  8, 11, 28,  9,  3,  8])

In [87]:
dfLaender['Count'] = count

In [88]:
sortedUniqueLandsPlus = ['Alle','Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 
                         'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 
                         'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for land in sortedUniqueLandsPlus:
    dfLaender[land] = 0

In [89]:
dfLaender

,Land,Count,Alle,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen
0,Baden-Württemberg,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bayern,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bremen,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hamburg,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hessen,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Niedersachsen,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Nordrhein-Westfalen,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Rheinland-Pfalz,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Saarland,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Schleswig-Holstein,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
for i,l in enumerate(sortedUniqueLands):
    print(l)
    for land in sortedUniqueLandsPlus:
        x = data200[(data200['Land'] == l) & (data200[land] == True)][land].count()
        dfLaender.set_value(i, land, x)

Baden-Württemberg
Bayern
Bremen
Hamburg
Hessen
Niedersachsen
Nordrhein-Westfalen
Rheinland-Pfalz
Saarland
Schleswig-Holstein


In [91]:
dfLaender

,Land,Count,Alle,Baden-Württemberg,Bayern,Berlin,Brandenburg,Bremen,Hamburg,Hessen,Mecklenburg-Vorpommern,Niedersachsen,Nordrhein-Westfalen,Rheinland-Pfalz,Saarland,Sachsen,Sachsen-Anhalt,Schleswig-Holstein,Thüringen
0,Baden-Württemberg,38,10,27,4,3,0,5,5,12,0,5,1,21,18,0,0,8,0
1,Bayern,58,14,18,44,16,0,5,7,15,0,10,8,24,21,0,0,8,0
2,Bremen,4,0,0,0,0,0,4,1,0,0,4,0,0,0,0,0,1,0
3,Hamburg,7,0,0,1,1,0,2,7,0,0,5,1,0,0,0,0,6,0
4,Hessen,8,1,0,0,1,0,1,5,7,0,4,3,1,0,0,0,6,0
5,Niedersachsen,11,1,0,0,5,0,9,8,6,0,10,5,2,1,0,0,9,0
6,Nordrhein-Westfalen,28,7,1,0,1,0,7,3,3,0,16,19,3,3,0,0,15,0
7,Rheinland-Pfalz,9,1,4,2,2,0,3,2,8,0,3,3,8,7,0,0,2,0
8,Saarland,3,0,1,0,0,0,1,0,0,0,0,0,2,3,0,0,0,0
9,Schleswig-Holstein,8,1,1,1,4,0,3,3,2,0,5,2,3,0,0,0,5,0


In [92]:
dfLaender.to_excel('Data/output/quickInfo1990.xlsx')